---
title: THEMIS Magnetic field data pipeline
---

- For convenience, we choose magnetic field data in **GSE** coordinate system
- The `fgs` data are in 3-4s resolution

## Loading data

In [4]:
from space_analysis.utils.speasy import Variables
from datetime import timedelta

In [10]:
timerange = ["2011-08-25", "2016-06-30"]
# timerange = ["2011-08-25", "2011-09-01"]
mission = "THB"
ts = timedelta(seconds=1)
tau = timedelta(seconds=60)

provider = 'archive/local'
mag_dataset = "THB_L2_FGM"
mag_parameters = ["thb_fgl_gse"]

mag_vars = Variables(
    provider = provider,
    dataset=mag_dataset,
    parameters=mag_parameters,
    timerange=timerange,
).retrieve_data()


KeyboardInterrupt: 

In [7]:
mag_vars.to_polars()

<LazyFrame [4 cols, {"Bx FGL-D": Float64 … "time": Datetime(time_unit='ns', time_zone=None)}] at 0x14F579210>

In [ ]:
# | export
def preprocess_data(
    raw_data: pl.LazyFrame,
    datatype: str = None,
) -> pl.LazyFrame:
    """
    Preprocess the raw dataset (only minor transformations)

    - Applying naming conventions for columns
    - Dropping duplicate time
    - Changing storing format to `parquet`

    """

    datatype = datatype.upper()
    name_mapping = {
        f"Bx {datatype}-D": "B_x",
        f"By {datatype}-D": "B_y",
        f"Bz {datatype}-D": "B_z",
    }

    return raw_data.sort("time").unique("time").rename(name_mapping)